In [17]:
import pandas as pd
import numpy as np
import scipy as sp
import queue as Q
import gurobipy as gp
from gurobipy import GRB
import itertools
import matplotlib.pyplot  as plt
import copy
import time
import collections
import os

def importlist(inputtext):
    list_of_lists = []

    with open(inputtext) as f:
        for line in f:
            inner_list = line.split("\n")
            doubleInner = [liner.strip() for liner in inner_list[0].split(" ")]
            if doubleInner[-1] == '':
                doubleInner.pop(-1)
            list_of_lists.append(doubleInner)
    flag = False
    holder = []
    objectives = []
    attributes = []
    for part in list_of_lists:
        part = list(map(int,part))
        if part[0] < 0:
            # it is an obj value
            objectives.append(part)
            flag = True
            continue
        if flag:
            # it is constraints
            attributes.append(part)
            continue
        #it is the first guys
        holder.append(part[0])
    itemCount = holder[0]
    b = holder[1:]
    return itemCount, np.array(b), np.array(objectives).reshape(-1,itemCount), np.array(attributes)

def  SolveKnapsack_21(Input_file_directory , method):
    n,b,c,a = importlist(Input_file_directory)
    direction = os.getcwd () + "/"
    if  method  == 1:
        holder = time.perf_counter()
        NDP, extra = BruteForce(c,a,b)
        time1 = time.perf_counter() - holder
        NDP = getordered(NDP)
        np.savetxt(direction + "BF_NDP_21.txt", NDP, delimiter='\t', newline='\n')
        summary = np.array([time1, NDP.shape[0], extra])
        np.savetxt(direction + "BF_SUMMARY_21.txt",summary, delimiter='\t', newline='\n')
    elif  method  == 2:
        holder = time.perf_counter()
        NDP, extra = rectangularSMART(c,a.transpose(),b)
        time2 = time.perf_counter() - holder
        NDP = getordered(NDP)
        np.savetxt(direction + "BB_NDP_21.txt", NDP, delimiter='\t', newline='\n')
        summary = np.array([time2, NDP.shape[0], extra])
        np.savetxt(direction + "BB_SUMMARY_21.txt",summary, delimiter='\t', newline='\n')
    elif  method  == 3:
        holder = time.perf_counter()
        NDP, extra = SupernalEff(c,a,b)
        time3 = time.perf_counter() - holder
        NDP = getordered(NDP)
        np.savetxt(direction + "SP_NDP_21.txt", NDP, delimiter='\t', newline='\n')
        summary = np.array([time3, NDP.shape[0], extra])
        np.savetxt(direction + "SP_SUMMARY_21.txt",summary, delimiter='\t', newline='\n')
    elif  method  == 4:
        holder = time.perf_counter()
        NDP, extra = rectangularSMART(c,a.transpose(),b)
        time4 = time.perf_counter() - holder
        NDP = getordered(NDP)
        np.savetxt(direction + "COMPETITION_2D_NDP_21.txt", NDP, delimiter='\t', newline='\n')
        summary = np.array([time4, NDP.shape[0], extra])
        np.savetxt(direction + "COMPETITION_2D_SUMMARY_21.txt",summary, delimiter='\t', newline='\n')
    elif  method  == 5:
        holder = time.perf_counter()
        NDP, extra = SupernalEff(c,a,b)
        time5 = time.perf_counter() - holder
        NDP = getordered(NDP)
        np.savetxt(direction + "COMPETITION_3D_NDP_21.txt", NDP, delimiter='\t', newline='\n')
        summary = np.array([time5, NDP.shape[0], extra])
        np.savetxt(direction + "COMPETITION_3D_SUMMARY_21.txt",summary, delimiter='\t', newline='\n')
    return
def getordered(NDP):
    NDP = np.array(NDP)
    setup = ()
    for i in reversed(range(NDP.shape[1])):
        setup += (NDP[:,i],)
    ind = np.lexsort(setup)
    return np.array([list(NDP[i,:]) for i in ind])
def rectangularSMART(c,a,b):
    
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    if m.status != 2:
        return [],1
    lexConstraint = round(m.objVal)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = round(m.objVal)
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = round(m.objVal)
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = round(m.objVal)
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
        if R[1][0] - R[0][0] == 1:
            continue
        if R[0][1] - R[1][1] == 1:
            continue
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = round(m.objVal)
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0],      (tempNW[0] - 1, ((R[0][1] + R[1][1])/2.0) + .001)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R3[1][1], name = 'bottom bound')
        
        m.optimize()
        
        z1=round(m.objVal)
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect
def SupernalEff(c,a,b):
        
    n = len(c[0])
    
    NDF = []
    
    temp_a = []
    for i in range(n):
        temp = []
        for j in range(len(a)):
            temp.append(a[j][i])
        temp_a.append(temp)
            
    c_orig = c.copy()
    J = len(c)
    temp_c = [item for sublist in c for item in sublist]
    c = np.array(temp_c)
    a = np.array(temp_a)
    b = np.array(b)
    
    cons = len(a[0])
    
    if J == 2:
        supernal = [0,0]
    
    elif J == 3:
        supernal = [0,0,0]
        
    regions1 = [supernal]
    regions2 = []
    
    m = gp.Model("Supernal Method")
    
    Items=list(np.arange(0,n))
    Dimensions= list(np.arange(0,cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")
    
    #Mute Output Text
    m.Params.OutputFlag = 0
        
    
    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))
        
    #Initital Supernal Region
    num_regions = 1
    m.addConstr(gp.quicksum(x[i]*c[i] for i in Items) <= 0, name = "z1")
    m.addConstr(gp.quicksum(x[i]*c[i+n] for i in Items) <= 0, name = "z2")
    
    
    if J == 3:
        l = [0.33,0.33,0.34]
        m.setObjective((gp.quicksum(x[i]*c[i] for i in Items)*l[0] + gp.quicksum(x[i]*c[i+n] for i in Items)*l[1] + gp.quicksum(x[i]*c[i+n*2] for i in Items)*l[2]), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[i+2*n] for i in Items) <= 0, name = "z3")
    
    elif J ==2:
        l = [0.3,0.7]
        m.setObjective((gp.quicksum(x[i]*c[i] for i in Items)*l[0] + gp.quicksum(x[i]*c[i+n] for i in Items)*l[1]), GRB.MINIMIZE)

    m.optimize()
    
    #Remove Denominated Regions for J = 3
    def deleteDominated(reg):
        nondom = reg.copy()
        for i in range(len(reg)):
            for j in range(len(reg)): 
    
                if i == j:
                    continue
    
                if reg[i][0]<= reg[j][0]:
                    if reg[i][1] <= reg[j][1]:
                        if reg[i][2] <= reg[j][2]:    
                            nondom.remove(reg[i])
                            break
        return nondom
    
    
    def optimize(region):
        
        m.getConstrByName('z1').RHS = region[0]
        m.getConstrByName('z2').RHS = region[1]
        
        if J == 3:
            m.getConstrByName('z3').RHS = region[2]
        
        m.optimize()
        
        Mat = np.zeros((1,n))
        i = 0
        j = 0
        
        try:
            for p in x:
                if j == Mat.shape[1]:
                    j = 0
                    i += 1
                Mat[i,j] = abs(x[p].x)
                j+=1
            
            z = []
            for i in range(J):
                z_temp = 0
                for j in range(n):
                    z_temp += c_orig[i][j]*Mat[0][j]
                z.append(z_temp)
            
            if z not in NDF:
                NDF.append(z)
            
            
            if J == 2:
    
                regions2.append([z[0]-1,region[1]])
                regions2.append([region[0],z[1]-1])        
                
            if J == 3:
                
                regions2.append([z[0]-1,region[1],region[2]])
                regions2.append([region[0],z[1]-1,region[2]])
                regions2.append([region[0],region[1],z[2]-1])                     
                
            return None
            
        except: 
            return None
        
    
    #While there are new regions to be processed in the next layer, continue
    while len(regions1) > 0:
        
        #iterate through all regions to be processed in current layer
        for r in range(len(regions1)):
            optimize(regions1[r])
        
        
        if J == 2:
            regions1 = regions2.copy()
        
        elif J == 3:
            #Remove dominated regions from next processing layer

            regions1 = deleteDominated(regions2)
        
        #Count new regions
        num_regions += len(regions1)
        #Reset empty list of regions for the next processing level
        regions2 = []
        
    ndf_final = NDF.copy() 
    if J == 2:

              
    
        for i in range(len(NDF)):
            for j in range(len(NDF)): 
    
                if i == j:
                    continue
        
                if NDF[i][0]>= NDF[j][0]:
                    if NDF[i][1] >= NDF[j][1]:
                        ndf_final.remove(NDF[i])
                    break
    
    return ndf_final, num_regions
def BruteForce(c,a,b):

    num = len(c[0])
    x_values = list(itertools.product([0, 1], repeat = num))
  
    feasible_z = []

    for x in range(len(x_values)):
        constraints_satisfied = 0
        for k in range(len(b)):
            if np.dot(x_values[x], a[k]) <= b[k]:
                constraints_satisfied += 1
        if constraints_satisfied == len(b): # only if all the constraints are satisfied
            obj = []
            for z in c:
                obj.append(np.dot(x_values[x], z))
          
            if obj not in feasible_z:
                feasible_z.append(obj)

    feasible_z = np.array(feasible_z)
    feasible_z = feasible_z[np.argsort(feasible_z[:, 0])]
    
    next_point = 0  # Next index in the NDP array to search for 

    while next_point < len(feasible_z):
        ndp = np.any(feasible_z < feasible_z[next_point], axis = 1)
        ndp[next_point] = True
        feasible_z = feasible_z[ndp]
        next_point = np.sum(ndp[:next_point]) + 1

    return feasible_z, 0